## Riksarkivet SBL  - Wikidata
version 0.1

WD egenskap [Property:P3217](https://www.wikidata.org/wiki/Property:P3217) 
* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/SBL.ipynb)  

In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", now)

Last run:  2020-09-13 10:43:35.394702


In [ ]:
import urllib3, json
import pandas as pd   
from bs4 import BeautifulSoup
import sys
import pprint
http = urllib3.PoolManager()

from tqdm.notebook import trange  
listNewItems =[]

for SBLnr in trange(5480,40000): 
    url = "https://sok.riksarkivet.se/sbl/Presentation.aspx?id=" + str(SBLnr)
    r = http.request('GET', url) 
    if len(r.data) > 11020:
        new_item = dict()
        #print(SBLnr)
        new_item['SBLnr'] = SBLnr
        soup = BeautifulSoup(r.data, "html.parser")  
        
        try:
            titel = soup.findAll("span", {"id": "ctl00_MainContent_lblTitle"})[0].get_text()
            new_item['titel'] = titel
            #print ("\t",titel)
        except:
            pass
        try:
            Fodd = soup.findAll("span", {"id": "ctl00_MainContent_lblFodelse"})[0].get_text()
            #print ("\t",Fodd)
            new_item['Fodd'] = Fodd
        except:
            pass
        try:
            Dod = soup.findAll("span", {"id": "ctl00_MainContent_lblDod"})[0].get_text()
            #print ("\t",Dod)
            new_item['Dod'] = Dod
        except:
            pass
        try:
            band = soup.findAll("span", {"id": "ctl00_MainContent_lblBandSida"})[0].get_text()
            #print ("\t",band)
            new_item['Band'] = band
            try:
#                year = band.get_text().split("(")[1].split(")")
                year = band.split("(")[1].split(")")[0]
                #print ("\t\tYear",year)
                new_item['Year'] = year
            except:
                pass
        except:
            pass
            
        listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")
        


In [ ]:
import csv
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SBL_newfile.csv", "w", newline='') as SBLfile:
        dict_writer = csv.DictWriter(SBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)

    dfSBLcsv = pd.read_csv("SBL_newfile.csv", sep=",")   
    
else:
    print ("Ingen fil skapas inga nya poster") 

In [ ]:
pd.set_option("display.max.rows", None) 

In [ ]:
dfSBLcsv.info()

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

querySKBL = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?sblid WHERE {
?item wdt:P3217 ?sblid
}"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDSBLtot = get_sparql_dataframe(endpoint_url, querySKBL)


In [ ]:
WDSBLtot.sort_index(inplace=True)  

In [ ]:
WDSBLtot.info()

In [ ]:
%matplotlib inline    
import matplotlib.pyplot as plt   
plotSBLYear = dfSBLcsv["Year"].value_counts()


In [ ]:
plotSBLYear.plot( kind = 'bar') 
plt.title("SBL artiklar publiserade per Bok/Band")
plt.show()

In [ ]:
dfSBLcsv.info()

In [ ]:
dfSBLcsv["url"] = "https://sok.riksarkivet.se/sbl/Presentation.aspx?id=" + str(dfSBLcsv["SBLnr"])

In [ ]:
#dfFamily = dfSBLAuthorcsv.dropna()  
dfPerson = dfSBLcsv.dropna(subset = ['Fodd', 'Dod'])

In [ ]:
dfPerson.info()

In [ ]:
dfFamily = dfSBLcsv[dfSBLcsv.isnull().any(1)]

In [ ]:
dfFamily.info()

In [ ]:
dfFamily.titel.str.contains("släkt").sum()

In [ ]:
dfFamily.titel.str.contains("Släkt").sum()

In [ ]:
dfFamily.Dod.str.contains("senas").sum()

In [ ]:
dfFamily[dfFamily.Dod.str.contains("senas", na=False)]

In [ ]:
dfFamily[dfFamily.Fodd.str.contains("", na=False)]

In [ ]:
#Check diff WDSBLtot and dfSBLcsv 
WDSBLtot.duplicated(subset=['wid']).sum()

In [ ]:
dfSBLcsv.duplicated(subset=['SBLnr']).sum()

In [ ]:
WDSBLtot.duplicated(subset=['sblid']).sum()

In [ ]:
WDSBLtot = WDSBLtot.rename(columns={'sblid':'SBLnr'})
WDSBLtot['SBLnr']= WDSBLtot['SBLnr'].astype(str).astype(int)
WDSBLtot.info()

In [ ]:
dfmerge = pd.merge(WDSBLtot, dfSBLcsv,how='outer', on='SBLnr',indicator=True)

In [ ]:
dfmerge["_merge"].value_counts()

In [ ]:
dfmerge

In [ ]:
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','WD_only').str.replace('right_only','SBL_only')


In [ ]:
WDSBLonly = dfmerge[dfmerge["_merge"] == "SBL_only"].copy()   
dfmerge["_merge"].value_counts()

In [ ]:
WDSBLonly.info()

In [ ]:
WDSBLonly.titel.str.contains("släkt").sum()

In [ ]:
#Inte släkt 
WDmissingPerson = WDSBLonly[~WDSBLonly.titel.str.contains("släkt")]

In [ ]:
WDmissingPerson.info()

In [ ]:
WDmissingPerson["Year"].value_counts()

In [ ]:
WDmissingPerson

In [ ]:
WDmissingPerson.info()

In [ ]:
#WDmissingPerson.to_json("SBL3.json")

In [ ]:
WDmissingPerson

In [ ]:
WDmissingPerson["Year"].value_counts()

In [ ]:
print("End run: ", datetime.now())